In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib
from mtcnn.mtcnn import MTCNN
from sklearn.preprocessing import LabelEncoder
detector = MTCNN()
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,ZeroPadding2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as k
import cv2

Using TensorFlow backend.


In [2]:
import os
os.chdir("D:\\MySQL\\image\\celeb_faces_images_train")
base_dir = os.getcwd()
base_dir

'D:\\MySQL\\image\\celeb_faces_images_train'

In [3]:
#create model
model = Sequential()
#add model layers
model.add(ZeroPadding2D((1,1),input_shape=(128,128, 3)))
#model.add(Convolution2D(64, (3,3), activation='relu', input_shape=(128,128,1)))
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Activation('softmax'))
#model.add(Dense(units=5,kernel_initializer='glorot_uniform',activation='softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 130, 130, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 66, 66, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       36992     
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0

In [5]:
classifier_model = load_model('5_cel.h5')

In [6]:
#Lets load the label encoder which was used during training
from sklearn.preprocessing import LabelEncoder
import numpy as np
le = LabelEncoder()
le.classes_ = np.load('class.npy')
le.classes_

array(['ben_afflek', 'elton_john', 'jerry_seinfeld', 'madonna',
       'mindy_kaling'], dtype='<U14')

In [7]:
import pickle
y = np.load('label.npy')
print(y)
y.shape

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


(89,)

In [9]:
le.inverse_transform(y)

array(['ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek',
       'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek',
       'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek',
       'ben_afflek', 'ben_afflek', 'elton_john', 'elton_john',
       'elton_john', 'elton_john', 'elton_john', 'elton_john',
       'elton_john', 'elton_john', 'elton_john', 'elton_john',
       'elton_john', 'elton_john', 'elton_john', 'elton_john',
       'elton_john', 'elton_john', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld',
       'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'madonna',
       'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna',
       'madonna', 'madonna', 'madonna'

In [10]:
classifier_model

In [15]:
#lets define the function to get embeddings for single image
def get_embedding_from_image(img):
    embedings = list()
    img = img_to_array(img)
    img1 = img.astype('float32')/255
    embedings.append(img1)
    a = np.asarray(embedings,dtype=np.float32)
    return(a)

In [37]:
from datetime import datetime
import cv2, pafy
from datetime import datetime
url = 'https://www.youtube.com/watch?v=m7zrrj-nD8s'
videoPafy = pafy.new(url)
best = videoPafy.getbest()
# Capture time duration of faces detected
faceFound = []
ts = [0,0,False,'Unknown']
vs = cv2.VideoCapture(0)
#vs = cv2.VideoCapture(best.url)
#vs = wbcamStream().start()
#dict_1 = {"Naved":[],"Zeeshan":[]}
count = 0
#face_label=list()
#final_db = list()
face_recognized = False
t_15_min = True

while True:
    check,frame = vs.read()
    count = count+1
    
    result = detector.detect_faces(frame)
    # in the result we will have multiple number of dictionaries for each image, and we need to extract the bounding box of the face.      
    if len(result):
        if ts[2] == False:
            ts[0] = datetime.now()
            ts[2] = True
        for i in range(len(result)):
            (startX,startY,endX,endY)=(result[i]["box"][0], result[i]["box"][1],result[i]["box"][0]+result[i]["box"][2], result[i]["box"][1] + result[i]["box"][3])
            confidence = result[i]["confidence"]
            if (confidence > 0.5):
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                if (startY >=0) and (endY >=0) and (startX >=0) and (endX >=0):
                    frame_face = frame[startY:endY, startX:endX]
                    frame_face=cv2.resize(frame_face,(128,128))
                    embedings=get_embedding_from_image(frame_face)
                    prob=classifier_model.predict_proba(embedings)[0].max()
                    if prob>=0.80:
                        label_name=le.inverse_transform(classifier_model.predict_classes(embedings))[0]
                        face_recognized=True
                        face_label.append(label_name)
                        cv2.putText(frame,label_name,(startX,startY),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                        ts[3] = label_name
                        found=1
                        break
    else:
        if ts[2] == True:
            ts[1] = datetime.now()
            ts[2] = False
            diff = ts[1]-ts[0]
            if diff.seconds!=0:
                faceFound.append((ts[3], diff.seconds+1, ts[0].strftime("%m/%d/%Y %H:%M:%S"), ts[1].strftime("%m/%d/%Y %H:%M:%S")))
                #for label in faceFound:
                    #dict_1[label].append((ts[3], diff.seconds+1, ts[0].strftime("%m/%d/%Y %H:%M:%S"), ts[1].strftime("%m/%d/%Y %H:%M:%S")))
                    
    cv2.imshow("web_frame",frame)
    if cv2.waitKey(1)==27:
        break
        
vs.release()
cv2.destroyAllWindows()
    
    
    
    


In [26]:
faceFound

[('elton_john', 13, '02/15/2020 08:07:05', '02/15/2020 08:07:18'),
 ('elton_john', 8, '02/15/2020 08:07:29', '02/15/2020 08:07:36'),
 ('elton_john', 3, '02/15/2020 08:07:41', '02/15/2020 08:07:44'),
 ('elton_john', 3, '02/15/2020 08:07:46', '02/15/2020 08:07:49'),
 ('elton_john', 2, '02/15/2020 08:07:51', '02/15/2020 08:07:52'),
 ('elton_john', 2, '02/15/2020 08:07:57', '02/15/2020 08:07:58'),
 ('elton_john', 8, '02/15/2020 08:08:03', '02/15/2020 08:08:10'),
 ('elton_john', 15, '02/15/2020 08:08:12', '02/15/2020 08:08:27'),
 ('madonna', 9, '02/15/2020 08:08:31', '02/15/2020 08:08:40')]

In [27]:
import pandas as pd
time_log=pd.DataFrame(faceFound,columns = ('Person Name','total time(sec)','entry time','exit time'))

In [28]:
time_log

,Person Name,total time(sec),entry time,exit time
0,elton_john,13,02/15/2020 08:07:05,02/15/2020 08:07:18
1,elton_john,8,02/15/2020 08:07:29,02/15/2020 08:07:36
2,elton_john,3,02/15/2020 08:07:41,02/15/2020 08:07:44
3,elton_john,3,02/15/2020 08:07:46,02/15/2020 08:07:49
4,elton_john,2,02/15/2020 08:07:51,02/15/2020 08:07:52
5,elton_john,2,02/15/2020 08:07:57,02/15/2020 08:07:58
6,elton_john,8,02/15/2020 08:08:03,02/15/2020 08:08:10
7,elton_john,15,02/15/2020 08:08:12,02/15/2020 08:08:27
8,madonna,9,02/15/2020 08:08:31,02/15/2020 08:08:40


In [29]:
import pandas as pd

In [34]:
pd.DataFrame.to_csv(time_log,'time_log.csv')

In [ ]:
from datetime import datetime
import cv2, pafy

vs = cv2.VideoCapture(0)
#vs = wbcamStream().start()
#dict_1 = {"Naved":[],"Zeeshan":[]}
count = 0
face_label=list()
final_db = list()


while True:
    check,frame = vs.read()
    count = count+1
    result = detector.detect_faces(frame)
    for i in range(len(result)):
        (startX,startY,endX,endY)=(result[i]["box"][0], result[i]["box"][1],result[i]["box"][0]+result[i]["box"][2], result[i]["box"][1] + result[i]["box"][3])
        confidence = result[i]["confidence"]
        if (confidence > 0.5):
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            if (startY >=0) and (endY >=0) and (startX >=0) and (endX >=0):
                frame_face = frame[startY:endY, startX:endX]
                frame_face=cv2.resize(frame_face,(128,128))
                embedings=get_embedding_from_image(frame_face)
                prob=classifier_model.predict_proba(embedings)[0].max()
                #prob=classifier_model.predict_proba(embedings)[0].max()
                if prob>=0.80:
                    label_name=le.inverse_transform(classifier_model.predict_classes(embedings))[0]
                    #face_recognized=True
                    #face_label.append(label_name)
                    cv2.putText(frame,label_name,(startX,startY),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)

                    #try:
                     #   cv2.imwrite(time_wise_face_path+"\\"+label_name+'\\'+str(time_detected).replace(" ","_").replace(":","_")+'.jpg',frame_face)
                    #except:
                     #   pass
                else:
                    face_recognized=False
    cv2.imshow("web_frame",frame)
    if cv2.waitKey(1)==27:
        break
        
vs.release()
cv2.destroyAllWindows()
    
    
    
    
